In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import nlppln
import ochre

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    wf.load(step_file='https://raw.githubusercontent.com/nlppln/ocrevaluation-docker/master/ocrevaluation.cwl')
    
    print wf.list_steps()
    
    archive = wf.add_input(data_file='File')
    ocr_dir_name = wf.add_input(ocr_dir_name='string', default='ocr')
    gs_dir_name = wf.add_input(gs_dir_name='string', default='gs')
    align_dir_name = wf.add_input(align_dir_name='string', default='align')
    de_dir_name = wf.add_input(de_dir_name='string', default='de')
    fr_dir_name = wf.add_input(fr_dir_name='string', default='fr')
    
    data_dir = wf.tar(in_file=archive)
    gs_de, gs_fr, ocr_de, ocr_fr = wf.sac2gs_and_ocr(in_dir=data_dir)
    
    # create alignments
    alignments_de, changes_de, metadata_de = wf.align_texts_wf(gs=gs_de, ocr=ocr_de)
    alignments_fr, changes_fr, metadata_fr = wf.align_texts_wf(gs=gs_fr, ocr=ocr_fr)
    
    # save files to correct dirs
    de_dir = wf.mkdir(dir_name=de_dir_name)
    fr_dir = wf.mkdir(dir_name=fr_dir_name)
    
    ocr_de_dir = wf.save_files_to_dir(dir_name=ocr_dir_name, in_files=ocr_de)
    ocr_de = wf.save_dir_to_subdir(inner_dir=ocr_de_dir, outer_dir=de_dir)
    gs_de_dir = wf.save_files_to_dir(dir_name=gs_dir_name, in_files=gs_de)
    gs_de = wf.save_dir_to_subdir(inner_dir=gs_de_dir, outer_dir=de_dir)
    
    ocr_fr_dir = wf.save_files_to_dir(dir_name=ocr_dir_name, in_files=ocr_fr)
    ocr_fr = wf.save_dir_to_subdir(inner_dir=ocr_fr_dir, outer_dir=fr_dir)
    gs_fr_dir = wf.save_files_to_dir(dir_name=gs_dir_name, in_files=gs_fr)
    gs_fr = wf.save_dir_to_subdir(inner_dir=gs_fr_dir, outer_dir=fr_dir)
    
    align_de_dir = wf.save_files_to_dir(dir_name=align_dir_name, in_files=alignments_de)
    align_de = wf.save_dir_to_subdir(inner_dir=align_de_dir, outer_dir=de_dir)
    
    align_fr_dir = wf.save_files_to_dir(dir_name=align_dir_name, in_files=alignments_fr)
    align_fr = wf.save_dir_to_subdir(inner_dir=align_fr_dir, outer_dir=fr_dir)
    
    wf.add_outputs(ocr_de=ocr_de)
    wf.add_outputs(gs_de=gs_de)
    wf.add_outputs(align_de=align_de)
    
    wf.add_outputs(ocr_fr=ocr_fr)
    wf.add_outputs(gs_fr=gs_fr)
    wf.add_outputs(align_fr=align_fr)
    
    wf.save(os.path.join(ochre.cwl_path(), 'sac-preprocess.cwl'), pack=True)